# Práctica 4

**Nombre:** Beatriz Domínguez Urías

**e-mail:** beatriz.dominguez0182@alumnos.udg.mx

## MODULES

In [7]:
import math
import numpy as np
import pandas as pd
import panel as pn
import panel.widgets as pnw

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

pn.extension("plotly", theme="dark")
pn.config.throttled = True
#pn.config.theme = "dark"

## CLASSES

In [8]:
class PlotUtility:
  """Class to simplify creation of plots with plotly graph objects module
  """
  def __init__(self, plot_title: str):
    self.plot_title = plot_title
    self.figure = go.Figure()

  def scatter_plot(self, plot_name: str, x, y, z=None, marker_size=2, line_size=2):
    if z is not None:
      self.figure.add_trace(go.Scatter3d(x = list(x),
                                         y = list(y),
                                         z = list(z),
                                         marker = dict(size=marker_size),
                                         line = dict(width=line_size),
                                         mode = "lines",
                                         name = plot_name,
                                         showlegend = True))
    else:
      self.figure.add_trace(go.Scatter(x = list(x),
                                       y = list(y),
                                       marker = dict(size=marker_size),
                                       line = dict(width=line_size),
                                       mode = "lines",
                                       name = plot_name,
                                       showlegend = True))

  def show_plot(self, xlabel:str=None, ylabel:str=None, zlabel:str=None):
    self.figure.update_layout(title = self.plot_title,
                              scene = dict(xaxis = dict(title= xlabel),
                                           yaxis = dict(title= ylabel),
                                           zaxis = dict(title= zlabel)))
    self.figure.show()
    
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [9]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Arguments:
    n_steps: descripcion1
    speed: descripcion2
    s_pos: descripcion3
  Returns:
    BM_2d_df: decripcion4
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

def correlated_random_walker(cauchy_exponent, n_steps=1000, speed=6, start_pos=[0,0]):
  """
  Returns a dataframe of a Correlated Random Walk following a cauchy
  distribution of cauchy_exponent with a given value of steps n_steps
  and speed that starts in position start_pos
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  # Init Correlated Random Walk dataframe
  CRW_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos":start_pos[0], "y_pos": start_pos[1]}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    # Define turn angle following a cauchy distribution
    turn_angle = wrapcauchy.rvs(cauchy_exponent)
    velocity = velocity.rotated(turn_angle)
    # Populate dataframe with walker displacement
    temp_df = pd.DataFrame([{"x_pos":CRW_2d_df.x_pos[i]+velocity.x, "y_pos": CRW_2d_df.y_pos[i]+velocity.y}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df

def generate_levy_flight(alpha, n_steps=10000, speed=1, miu=0, beta=1.0, cauchy_exponent=0.7, start_pos=[0,0]):
  """
  Returns a dataframe of a Levy Flight following a cauchy distribution for the
  turning angle and a levy stable distribution for the displacement
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  # Init Lévy Flight dataframe
  levy_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos":start_pos[0], "y_pos": start_pos[1]}])
  levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  # Init steps control variables
  levy_steps = 1
  step_count = 0

  for i in range(n_steps-1):
    if step_count >= levy_steps:
      # Define turn angle following a cauchy distribution
      turn_angle = wrapcauchy.rvs(cauchy_exponent)
      velocity = velocity.rotated(turn_angle)

      # Populate dataframe with walker displacement
      temp_df = pd.DataFrame([{"x_pos":levy_df.x_pos[i]+velocity.x, "y_pos": levy_df.y_pos[i]+velocity.y}])
      levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

      # Reset steps control variables
      levy_steps = levy_stable.rvs(alpha=alpha, beta=beta, loc=miu)
      step_count = 0
    else:
      # Increase a step without a change on direction
      step_count += 1
      temp_df = pd.DataFrame([{"x_pos":levy_df.x_pos[i]+velocity.x, "y_pos": levy_df.y_pos[i]+velocity.y}])
      levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

  return levy_df

def get_euclidean_distance(point_one: pd.Series, point_two: pd.Series) -> float:
  """_summary_

  :param point_one: _description_
  :type point_one: pd.Series
  :param point_two: _description_
  :type point_two: pd.Series
  :return: _description_
  :rtype: float
  """    
  x_comp = math.pow(point_two.iloc[0] - point_one.iloc[0],2)
  y_comp = math.pow(point_two.iloc[1] - point_one.iloc[1],2)
  distance = math.sqrt(x_comp + y_comp)
  
  return distance

def get_path_length(trajectory: pd.DataFrame) -> pd.DataFrame:
  """_summary_

  :param trajectory: _description_
  :type trajectory: pd.DataFrame
  :return: _description_
  :rtype: pd.DataFrame
  """  
  path_length_df = pd.DataFrame(columns = ["distance"])
  for i in range(1, trajectory.shape[0]):
      temp_df = pd.DataFrame([{"distance":get_euclidean_distance(trajectory.iloc[i-1],trajectory.iloc[i])}])
      path_length_df = pd.concat([path_length_df, temp_df], ignore_index=True)

  return path_length_df.cumsum()

def get_mean_squared_displacement(trajectory: pd.DataFrame) -> pd.DataFrame:
  """_summary_

  :param trajectory: _description_
  :type trajectory: pd.DataFrame
  :return: _description_
  :rtype: pd.DataFrame
  """  
  N = trajectory.shape[0]
  MSD_df = pd.DataFrame(columns = ["MSD"])

  for tau in range(1,N):
      displacement_vec = np.array([math.pow(get_euclidean_distance(trajectory.iloc[i-tau], trajectory.iloc[i]), 2)
                              for i in range(tau, BM_2d_df_6.shape[0],1)])
      temp_df = pd.DataFrame([{"MSD": np.mean(displacement_vec)}])
      MSD_df = pd.concat([MSD_df, temp_df], ignore_index=True)
      displacement_vec = np.empty(shape=(0))
  
  return MSD_df

## DASHBOARD

### Widgets for parameters

In [10]:
widg_traj = pnw.RadioButtonGroup(name="Trajectory type", options=["BM", "CRW", "LF"], width=10)
widg_steps = pnw.EditableIntSlider(name="Number of steps", value=100, step=10, start=1, end=10000, width=235)
widg_x_pos = pnw.IntInput(name="Start pos of X", value=0, step=1, start=-100, end=100, width=100)
widg_y_pos = pnw.IntInput(name="Start pos of Y", value=0, step=1, start=-100, end=100, width=100)
widg_speed = pnw.EditableIntSlider(name="Speed", value=6, step=1, start=1, end=100, width=235)
widg_cauchy = pnw.FloatInput(name="Cauchy exponent", value=0, step=0.1, start=0.1, end=2, width=100)
widg_levy_alpha = pnw.FloatInput(name="Levy exponent", value=0, step=0.1, start=0.1, end=2, width=100)
widg_levy_beta = pnw.FloatInput(name="Beta", value=0, step=0.1, start=0.1, end=2, width=100)
widg_levy_loc = pnw.IntInput(name="Loc", value=0, step=1, start=-100, end=100, width=100)
widg_metric = pnw.Select(name="Metric type", options=["PL", "MSD"], width=100)

### Interactive Functions

In [21]:
def update_layout(value:str) -> pn.layout.Column:
    """_summary_

    :param value: _description_
    :type value: str
    :return: _description_
    :rtype: pn.layout.Column
    """    
    pos_row = pn.Row(widg_x_pos, widg_y_pos)
    base_column = pn.Column("### Parameters", pn.Row(widg_steps), pos_row, pn.Row(widg_speed))#, styles=dict(background='WhiteSmoke'))
    
    if value == "BM":
        updated_column = base_column
    elif value == "CRW":
        updated_column = pn.Column(base_column, pn.Row(widg_cauchy))#, styles=dict(background='WhiteSmoke'))
    elif value == "LF":
        updated_column = pn.Column(base_column, pn.Row(widg_cauchy, widg_levy_alpha), pn.Row(widg_levy_beta, widg_levy_loc))#, styles=dict(background='WhiteSmoke'))
    
    return updated_column

### Layout

In [22]:
def test_plot_metric(value):
    return pn.Column(f"Plot for {value}", go.Figure())

param_column = pn.Column("## Trajectory type", widg_traj, pn.bind(update_layout, widg_traj))#, styles=dict(background='WhiteSmoke'))
traj_column = pn.Column("## Trajectory plot", pn.pane.Plotly(go.Figure()), sizing_mode="stretch_width")#, styles=dict(background='WhiteSmoke'))
metric_column = pn.Column("## Metric plot", widg_metric, pn.bind(test_plot_metric,widg_metric), sizing_mode="stretch_width")#, styles=dict(background='WhiteSmoke'))

layout = pn.Row(param_column, traj_column, metric_column)
layout

Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='LF', width=10)
        [2] ParamFunction(function, _pane=Column, defer_load=False)
    [1] Column(sizing_mode='stretch_width')
        [0] Markdown(str)
        [1] Plotly(Figure)
    [2] Column(sizing_mode='stretch_width')
        [0] Markdown(str)
        [1] Select(name='Metric type', options=['PL', 'MSD'], value='PL', width=100)
        [2] ParamFunction(function, _pane=Column, defer_load=False)